In [1]:
import os
import cv2
import argparse
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from tqdm import tqdm
from sklearn.cluster import DBSCAN
from imutils.video import FileVideoStream
np.random.seed(42)
pd.options.mode.chained_assignment = None  # default='warn'
np.seterr(divide='ignore') # ignore divide by zero when calculating angle
%load_ext line_profiler

/home/martin/.local/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [ ]:
# parameters for Hough Line detection
rho = 1                 # distance resolution in pixels of the Hough grid
theta = np.pi / 180     # angular resolution in radians of the Hough grid
threshold = 40          # minimum number of votes (intersections in Hough grid cell)
min_line_length = 25    # minimum number of pixels making up a line
max_line_gap = 10       # maximum gap in pixels between connectable line segments
WIDTH = 720             # width to resize the processed video to

def resize(img):
    return cv2.resize(img, (WIDTH, WIDTH))

def process_video(fname, save_video=False, savename=None, show_video=False, save_stats=False,
                    frame_limit=False):
    if savename == None:
        savename = "saber_tracking.avi"

    if save_video:
        # Initialize video writer to save the results
        out = cv2.VideoWriter(savename, cv2.VideoWriter_fourcc(*'XVID'), 30.0, 
                                 (WIDTH, WIDTH), True)

    cap = cv2.VideoCapture(fname)
    ret, frame = cap.read()
    total_frames = 500 if frame_limit else int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    pbar = tqdm(total=total_frames)
    frame_num = 0

    output_path = savename.replace(".avi", "_data.parquet")
    data = {"frame" : [],
            "centroid_x" : [],
            "centroid_y" : [],
            "angle" : [],
            "length" : []}

    # instantiate DBSCAN for use throughout
    # n_jobs parallelisation introduces too much overhead
    db = DBSCAN(eps=5, min_samples=2)

    while ret:
        ret, frame = cap.read()
        # these channels were swapped in the notebook
        b = (frame[:, :, 2] > 200).astype(int)
        r = (frame[:, :, 0] > 220).astype(int)

        # convert to HSV for more masking options
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        v = (hsv[:, :, 2] > 210).astype(int)
        s = cv2.inRange(hsv[:, :, 1],  140, 175)

        # combine masks into one
        m1 = np.logical_and(b, s)
        m2 = np.logical_and(r, v)
        mask = (m1 + m2).astype(np.uint8)

        # Run Hough on edge detected image
        # Output "lines" is an array containing endpoints of detected line segments
        lines = cv2.HoughLinesP(mask, rho, theta, threshold, np.array([]), min_line_length, max_line_gap)

        # process lines
        if isinstance(lines, np.ndarray):
            for line in lines:
                x1, y1, x2, y2 = line.ravel()                   
                centroid = (int((x1 + x2) / 2), int((y1 + y2) / 2))
                x_diff = x1 - x2
                y_diff = y1 - y2
                length = (x_diff * x_diff + y_diff * y_diff) ** 0.5
                edge_x = 200 < centroid[0] < 1080
                edge_y = 100 < centroid[1] < 620
                if 100 > length > 30 and edge_x and edge_y: # length of 25 or 30
                    degrees = np.rad2deg(np.arctan(y_diff / x_diff))
                    data["frame"].append(frame_num)
                    data["centroid_x"].append(centroid[0])
                    data["centroid_y"].append(centroid[1])
                    data["angle"].append(degrees)
                    data["length"].append(length)

        # perform clustering to reduce data
        df = pd.DataFrame(data)
        if df.shape[0] > 0:
            db.fit(df[["centroid_x", "centroid_y", "angle"]])
            df["labels"] = db.labels_
            df = df.query("labels != -1")
            if df.shape[0] > 0:
                df = df.groupby(["frame", "labels"], as_index=False)[["centroid_x", "centroid_y", "angle"]].mean()
                for centroid in df[["centroid_x", "centroid_y"]].values:
                    cv2.drawMarker(frame, centroid.astype(int), (0, 255, 0), markerType=cv2.MARKER_CROSS, thickness=2)

        if save_stats:
            # Create a parquet table from your dataframe
            table = pa.Table.from_pandas(df)

            # Write direct to your parquet file
            pq.write_to_dataset(table, root_path=output_path)
            
            # reset data structure
            data = {"frame" : [],
                    "centroid_x" : [],
                    "centroid_y" : [],
                    "angle" : [],
                    "length" : []}

        resized = resize(frame)
        
        if show_video:
            cv2.imshow("Frame", resized)
        if save_video:
            out.write(resized)
        frame_num += 1
        pbar.update(1)

        key = cv2.waitKey(1)
        if key == ord('q'):
            break
        if key == ord('p'):
            cv2.waitKey(-1) # wait until any key is pressed
        if frame_limit and frame_num == 500:
            break

    cap.release()
    if save_video:
        out.release()
    if show_video:
        cv2.destroyAllWindows()

In [ ]:
%lprun -f process_video process_video("test_video.mp4", True, "min_length_30.avi", False, True, True)

Base function took 62 seconds to run on 500 frames (8 FPS) when `save_stats=False`; only 17 seconds when `save_stats=True` (29.4 FPS). Odd behaviour indeed.

Results when saving stats:
* 3.1 seconds (18.8%) masking with numpy
* 0.821 seconds (5.1%) processing lines
* 5.8 seoncds (36.8%) on clustering and DataFrame reduction
* 0.75 seconds (4.7%) making a Parquet table and writing to file
* The rest taken by optimized OpenCV functions

These processes account for ~62% of the time taken. I think it may be possible to reduce total time taken by a third

### Optimizations to implement
Use threading as per [link](https://pyimagesearch.com/2017/02/06/faster-video-file-fps-with-cv2-videocapture-and-opencv/) to queue frames for reading and theoretically increase FPS by ~50% (according to article). This [article](https://towardsdatascience.com/lightning-fast-video-reading-in-python-c1438771c4e6) may also be of use in increasing read speed. Alternatively, multiprocessing may work as in this [blog post](https://xailient.com/blog/parallel-processing-using-python-for-faster-video-processing/) since it also covers writing the frames as well

In [2]:
from sklearnex.cluster import DBSCAN

# parameters for Hough Line detection
RHO = 1                 # distance resolution in pixels of the Hough grid
THETA = np.pi / 180     # angular resolution in radians of the Hough grid
THRESHOLD = 20          # minimum number of votes (intersections in Hough grid cell)
MIN_LINE_LENGTH = 5    # minimum number of pixels making up a line
MAX_LINE_GAP = 2       # maximum gap in pixels between connectable line segments
WIDTH = 720             # width to resize the processed video to

def optimized(fname, save_video=False, savename=None, show_video=False, save_stats=False,
                    frame_limit=False):
    if savename == None:
        savename = "saber_tracking.avi"

    if save_video:
        # Initialize video writer to save the results
        out = cv2.VideoWriter(savename, cv2.VideoWriter_fourcc(*'XVID'), 30.0, 
                                 (WIDTH, WIDTH), True)

    cap = cv2.VideoCapture(fname)
    ret, frame = cap.read()
    total_frames = 500 if frame_limit else int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    pbar = tqdm(total=total_frames)
    frame_num = 0

    output_path = savename.replace(".avi", "_data.csv")
    # prevent appending to existing file
    if os.path.exists(output_path):
        os.remove(output_path)
    data = {"frame" : [],
            "centroid_x" : [],
            "centroid_y" : [],
            "angle" : [],
            "length" : []}

    # instantiate DBSCAN for use throughout
    # n_jobs parallelisation introduces too much overhead
    db = DBSCAN(eps=5, min_samples=2)

    while ret:
        ret, frame = cap.read()
        if ret:
            frame = cv2.resize(frame, (frame.shape[1] // 2, frame.shape[0] // 2))
            # these channels were swapped in the notebook
            b = cv2.inRange(frame[:, :, 2], 200, 255)
            r = cv2.inRange(frame[:, :, 0], 180, 255)

            # convert to HSV for more masking options
            hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
            v = cv2.inRange(hsv[:, :, 2], 170, 255)
            s = cv2.inRange(hsv[:, :, 1], 140, 175)

            # combine masks into one
            m1 = cv2.bitwise_and(b, s)
            m2 = cv2.bitwise_and(r, v)
            mask = cv2.bitwise_or(m1, m2)

            # Run Hough on masked image
            # Output "lines" is an array containing endpoints of detected line segments
            lines = cv2.HoughLinesP(mask, RHO, THETA, THRESHOLD, np.array([]), MIN_LINE_LENGTH, MAX_LINE_GAP)

            # process lines
            if isinstance(lines, np.ndarray):
                for line in lines:
                    x1, y1, x2, y2 = line.ravel()
                    centroid = (int((x1 + x2) / 2), int((y1 + y2) / 2))
                    x_diff = x1 - x2
                    y_diff = y1 - y2
                    length = (x_diff * x_diff + y_diff * y_diff) ** 0.5
                    edge_x = 100 < centroid[0] < 540
                    edge_y = 50 < centroid[1] < 310
                    l = 50 > length > 10
                    if l and edge_x and edge_y: # length of 25 or 30
                        degrees = np.rad2deg(np.arctan(y_diff / x_diff))
                        data["frame"].append(frame_num)
                        data["centroid_x"].append(centroid[0])
                        data["centroid_y"].append(centroid[1])
                        data["angle"].append(degrees)
                        data["length"].append(length)

            # perform clustering to reduce data
            df = pd.DataFrame(data)
            if df.shape[0] > 0:
                df["labels"] = db.fit(df[["centroid_x", "centroid_y", "angle"]].values).labels_
                df = df[(df != -1).all(axis=1)]
                if df.shape[0] > 0:
                    df = df.groupby("labels", as_index=False, sort=False).mean()
                    for cx, cy in zip(df["centroid_x"], df["centroid_y"]):
                        cv2.drawMarker(frame, (int(cx), int(cy)), (0, 255, 0), markerType=cv2.MARKER_CROSS, thickness=2)

            if save_stats:
                df.to_csv(output_path, index=False, mode='a',
                            header=not os.path.exists(output_path))

                # reset data structure
                data = {"frame" : [],
                        "centroid_x" : [],
                        "centroid_y" : [],
                        "angle" : [],
                        "length" : []}

            resized = cv2.resize(frame, (WIDTH, WIDTH))

            if show_video:
                cv2.imshow("Frame", frame)
            if save_video:
                out.write(resized)
            frame_num += 1
            pbar.update(1)

            key = cv2.waitKey(1)
            if key == ord('q'):
                break
            if key == ord('p'):
                cv2.waitKey(-1) # wait until any key is pressed
            if frame_limit and frame_num == 500:
                break

    cap.release()
    if save_video:
        out.release()
    if show_video:
        cv2.destroyAllWindows()

In [3]:
# uses same default params as 

def optimized_fvs(fname, save_video=False, savename=None, show_video=False, save_stats=False,
                    frame_limit=False):
    if savename == None:
        savename = "saber_tracking.avi"

    if save_video:
        # Initialize video writer to save the results
        out = cv2.VideoWriter(savename, cv2.VideoWriter_fourcc(*'XVID'), 30.0, 
                                 (WIDTH, WIDTH), True)

    # use OpenCV to get total frames
    cap = cv2.VideoCapture(fname)
    total_frames = 500 if frame_limit else int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.release()
    
    pbar = tqdm(total=total_frames)
    frame_num = 0

    output_path = savename.replace(".avi", "_data.csv")
    # prevent appending to existing file
    exists = os.path.exists(output_path)
    if exists:
        os.remove(output_path)
    data = {"frame" : [],
            "centroid_x" : [],
            "centroid_y" : [],
            "angle" : [],
            "length" : []}

    # instantiate DBSCAN for use throughout
    # n_jobs parallelisation introduces too much overhead
    db = DBSCAN(eps=5, min_samples=2)
    
    # start imutils streaming
    fvs = FileVideoStream(fname, queue_size=250).start()

    while fvs.more():
        frame = fvs.read()
        if isinstance(frame, np.ndarray):
            frame = cv2.resize(frame, (frame.shape[1] // 2, frame.shape[0] // 2))
            # these channels were swapped in the notebook
            b = cv2.inRange(frame[:, :, 2], 200, 255)
            r = cv2.inRange(frame[:, :, 0], 180, 255)

            # convert to HSV for more masking options
            hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
            v = cv2.inRange(hsv[:, :, 2], 170, 255)
            s = cv2.inRange(hsv[:, :, 1], 140, 175)

            # combine masks into one
            m1 = cv2.bitwise_and(b, s)
            m2 = cv2.bitwise_and(r, v)
            mask = cv2.bitwise_or(m1, m2)

            # Run Hough on masked image
            # Output "lines" is an array containing endpoints of detected line segments
            lines = cv2.HoughLinesP(mask, RHO, THETA, THRESHOLD, np.array([]), MIN_LINE_LENGTH, MAX_LINE_GAP)

            # process lines
            if isinstance(lines, np.ndarray):
                for line in lines:
                    x1, y1, x2, y2 = line.ravel()
                    centroid = (int((x1 + x2) / 2), int((y1 + y2) / 2))
                    x_diff = x1 - x2
                    y_diff = y1 - y2
                    length = (x_diff * x_diff + y_diff * y_diff) ** 0.5
                    edge_x = 100 < centroid[0] < 540
                    edge_y = 50 < centroid[1] < 310
                    l = 50 > length > 10
                    if l and edge_x and edge_y: # length of 25 or 30
                        degrees = np.rad2deg(np.arctan(y_diff / x_diff))
                        data["frame"].append(frame_num)
                        data["centroid_x"].append(centroid[0])
                        data["centroid_y"].append(centroid[1])
                        data["angle"].append(degrees)
                        data["length"].append(length)

            # perform clustering to reduce data
            df = pd.DataFrame(data)
            if df.shape[0] > 0:
                df["labels"] = db.fit(df[["centroid_x", "centroid_y", "angle"]].values).labels_
                df = df[(df != -1).any(axis=1)]
                if df.shape[0] > 0:
                    df = df.groupby("labels", as_index=False, sort=False).mean()
                    for cx, cy in zip(df["centroid_x"], df["centroid_y"]):
                        cv2.drawMarker(frame, (int(cx), int(cy)), (0, 255, 0), markerType=cv2.MARKER_CROSS, thickness=2)

            if save_stats:
                df.to_csv(output_path, mode='a', header=not exists)

                # reset data structure
                data = {"frame" : [],
                        "centroid_x" : [],
                        "centroid_y" : [],
                        "angle" : [],
                        "length" : []}

            resized = cv2.resize(frame, (WIDTH, WIDTH))

            if show_video:
                cv2.imshow("Frame", frame)
            if save_video:
                out.write(resized)
            frame_num += 1
            pbar.update(1)

            key = cv2.waitKey(1)
            if key == ord('q'):
                break
            if key == ord('p'):
                cv2.waitKey(-1) # wait until any key is pressed
            if frame_limit and frame_num == 500:
                break

    fvs.stop()
    if save_video:
        out.release()
    if show_video:
        cv2.destroyAllWindows()

In [ ]:
%lprun -f optimized optimized("test_video.mp4", True, "min_length_30.avi", False, True, False)

In [4]:
%lprun -f optimized_fvs optimized_fvs("test_video.mp4", True, "min_length_30.avi", False, True, False)

100%|█████████████████████████████████████| 33838/33838 [12:01<00:00, 46.92it/s]


In [ ]:
help(FileVideoStream)

## Optimizations
* Removed resize function and included in main function body
* Replacing Parquet with Pandas CSV reduces stats saving time from 0.75 to 0.375 seconds
* Replacing numpy masking with OpenCV functions reduces masking time from 3.1 to 0.993 seconds!
* Replacing Pandas query with `df[df[col] != value]` reduces time from 1.7 to 0.452 seconds!
* Removing the `frame` column as an index (since it's constant) reduces time from 1.7 to 1.2 seconds
* Fitting DBSCAN with `.values` rather than a `DataFrame` reduces time from 1.73 to 1.14 seconds
* Using `sklearnex` reduced DBSCAN fitting time from 1.52 to 1.14 seconds

## Results
* Base function ran for 17.3986 seconds
* Optimized function ran for 11.5512 seconds
* <strong>33.6% reduction in time to process 500 frames</strong>

In [ ]:
df = pd.read_csv("min_length_30_data.csv")

Optimization taken from this StackOverflow [answer](https://stackoverflow.com/questions/66292101/fastest-ways-to-filter-for-values-in-pandas-dataframe), which runs almost twice as fast as `df[df["labels"] != -1]`

In [ ]:
%timeit df[(df != -1).all(axis=1)]

## Examine optimized profile

In [2]:
def read_profile_as_df(fname):
    with open(fname, "r") as f:
        lines = f.readlines()
    arr = []
    for l in lines:
        if l.startswith("   "):
            split = [i.strip() for i in l.split("   ") if i.strip() != ""]
            if len(split) == 6:
                arr.append(split)
        elif l.startswith("Line"):
            columns = [i.strip() for i in l.split("  ") if i != ""]
    prof = pd.DataFrame(arr, columns=columns)
    # set correct data types
    prof = prof.astype({"Line #" : "uint16", "Hits" : "uint16",
            "Time" : "float64", 
             "Per Hit" : "float16",
            "% Time" : "float16"})
    return prof

In [5]:
prof = read_profile_as_df("profiles/optimized_unlimited_profile.txt")

top = 10
perc = 1
largest = prof["% Time"].nlargest(10)
sub = prof.iloc[largest[largest >= perc].index, :]
print(f"Of the top {top} most time-consuming lines,")
print(f"{sub.shape[0]} are >= {perc} % for {sub['% Time'].sum():.1f} % of the time taken")
sub

Of the top 10 most time-consuming lines,
10 are >= 1 % for 63.0 % of the time taken


,Line #,Hits,Time,Per Hit,% Time,Line Contents
50,88,31047,79896179.0,2574.000000,12.601562,"df = df.groupby(""labels"", as_index=False, sort..."
27,61,33837,73093371.0,2160.000000,11.601562,"lines = cv2.HoughLinesP(mask, RHO, THETA, THRE..."
47,85,33711,71461387.0,2120.000000,11.296875,"df[""labels""] = db.fit(df[[""centroid_x"", ""centr..."
48,86,33711,40139487.0,1191.000000,6.300781,df = df[(df != -1).all(axis=1)]
16,42,33838,34740119.0,1027.000000,5.500000,"ret, frame = cap.read()"
45,83,33837,27722900.0,819.500000,4.398438,df = pd.DataFrame(data)
54,93,2138,25042863.0,370.000000,4.000000,"df.to_csv(output_path, index=False, mode='a',"
31,67,56858,21365348.0,5.398438,3.400391,"centroid = (int((x1 + x2) / 2), int((y1 + y2) ..."
61,103,33837,12410802.0,366.750000,2.000000,"resized = cv2.resize(frame, (WIDTH, WIDTH))"
34,70,56858,11843659.0,3.000000,1.900391,length = (x_diff * x_diff + y_diff * y_diff) *...


In [6]:
prof = read_profile_as_df("profiles/optimized_fvs_unlimited_profile.txt")

largest = prof["% Time"].nlargest(10)
sub = prof.iloc[largest[largest >= perc].index, :]
print(f"Of the top {top} most time-consuming lines,")
print(f"{sub.shape[0]} are >= {perc} % for {sub['% Time'].sum():.1f} % of the time taken")
sub

Of the top 10 most time-consuming lines,
10 are >= 1 % for 62.5 % of the time taken


,Line #,Hits,Time,Per Hit,% Time,Line Contents
53,86,33712,91085206.0,2702.000000,14.101562,"df = df.groupby(""labels"", as_index=False, sort..."
30,59,33838,75434282.0,2230.000000,11.601562,"lines = cv2.HoughLinesP(mask, RHO, THETA, THRE..."
50,83,33712,74842692.0,2220.000000,11.601562,"df[""labels""] = db.fit(df[[""centroid_x"", ""centr..."
51,84,33712,44580025.0,1322.000000,6.898438,df = df[(df != -1).any(axis=1)]
57,91,33838,29039414.0,858.000000,4.500000,"df.to_csv(output_path, mode='a', header=not ex..."
48,81,33838,28808897.0,851.500000,4.398438,df = pd.DataFrame(data)
34,65,56962,22308489.0,5.601562,3.400391,"centroid = (int((x1 + x2) / 2), int((y1 + y2) ..."
63,100,33838,13553239.0,400.500000,2.099609,"resized = cv2.resize(frame, (WIDTH, WIDTH))"
21,42,33838,12889584.0,381.000000,2.000000,"frame = cv2.resize(frame, (frame.shape[1] // 2..."
37,68,56962,12489243.0,3.099609,1.900391,length = (x_diff * x_diff + y_diff * y_diff) *...
